# Building AI Agents with LangGraph and LLMs

This workshop guides you through creating an AI agent using LangGraph, a powerful framework for building stateful, reasoning-focused AI systems with explicit control flows.

## What are AI Agents?

AI agents are autonomous systems that perceive their environment, make decisions, and take actions to accomplish specific goals. Unlike simple language models that just respond to prompts, agents can:

1. **Plan and Reason**: Break down complex tasks into logical steps
2. **Use Tools**: Access external capabilities like search engines, databases, or APIs
3. **Maintain State**: Remember context and previous actions across interactions
4. **Make Decisions**: Choose appropriate actions based on their reasoning

## What is LangGraph?

LangGraph is a framework for building stateful, reasoning-focused AI systems with explicit control flows. It provides:

1. **Structured Reasoning**: Create explicit multi-step reasoning processes
2. **State Management**: Maintain and transform context between steps
3. **Directed Workflows**: Design clear execution paths between components
4. **Conditional Logic**: Implement decision-making capabilities

In this workshop, we'll build a web search agent to demonstrate these concepts in action.

In [8]:
import os
from typing import TypedDict, List, Annotated, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Literal, Optional, Union
from langchain_community.document_loaders import WebBaseLoader
import json
# Load environment variables from .env file
load_dotenv()

# LangGraph imports
from langgraph.graph import StateGraph, END

# LangChain imports
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI  # Import Gemini
from langchain_openai import ChatOpenAI  # Alternative: OpenAI
from langchain_core.prompts import ChatPromptTemplate

# For web search
from langchain_community.utilities import GoogleSearchAPIWrapper  # Option 1
from langchain_community.utilities import SerpAPIWrapper  # Option 2

## Setting Up the Environment

We'll start by importing the necessary libraries for our agent. These include:

- **LangGraph**: For creating our agent's reasoning workflow
- **LangChain**: For working with LLMs and tools
- **Pydantic**: For creating strongly typed data models
- **Web Search APIs**: To give our agent the ability to search the internet

In [9]:
# Check if API keys are set
print("Google API key is", "set" if os.environ.get("GOOGLE_API_KEY") else "not set")
print("OpenAI API key is", "set" if os.environ.get("OPENAI_API_KEY") else "not set")
print("Google CSE ID is", "set" if os.environ.get("GOOGLE_CSE_ID") else "not set")
print("SerpAPI key is", "set" if os.environ.get("SERPAPI_API_KEY") else "not set")

Google API key is set
OpenAI API key is not set
Google CSE ID is set
SerpAPI key is not set


## Checking API Keys

Before proceeding, we need to verify that the necessary API keys are available in our environment:

- **Google API Key**: For Gemini LLM access or Google search capabilities
- **OpenAI API Key**: Alternative LLM provider
- **Google CSE ID**: For Google Custom Search Engine
- **SerpAPI Key**: Alternative search engine API

The agent will use these APIs to access knowledge beyond its training data.

In [10]:
class SearchResult(BaseModel):
    id: int
    title: str
    snippet: str 
    link: str
    relevance_score: Optional[float] = None
    full_content: Optional[str] = None
    
class AgentState(BaseModel):
    messages: List[Union[HumanMessage, AIMessage, SystemMessage]] = Field(description="The chat history")
    search_results: List[SearchResult] = Field(default_factory=list, description="The results from the web search")
    optimized_query: Optional[str] = Field(default=None, description="The LLM-optimized search query")
    
    class Config:
        arbitrary_types_allowed = True

In [11]:
def answer_question(state: AgentState):
    user_query = next(
        (m.content for m in reversed(state.messages)
         if isinstance(m, HumanMessage)),
        ""
    )

    answer_prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a knowledgeable basketball analyst.\n"
         "• Always give concrete data you *know* (dates, scores, venues).\n"
         "• If you’re not sure of the newest result, cite the last result you’re sure about.\n"
         "• Include the actual score for any game you reference."),
        ("human", "Answer this as specifically as possible: {query}")
    ])

    answer = llm.invoke(
        answer_prompt.format_messages(query=user_query)
    ).content.strip()

    # Because messages has an `add_messages` reducer we can just
    # return the *new* list element and LangGraph will append it.
    return {"messages": [AIMessage(content=answer)]}


## Basic Agent Implementation

Here we implement our first agent function that analyzes user questions and provides responses.

### Understanding Node Functions in LangGraph

In LangGraph, **nodes** are functions that:
1. Take the current state as input
2. Perform some processing or reasoning
3. Return a modified state as output

Our `analyze_question` function implements this pattern by:
1. Extracting the user's message from the state
2. Creating a prompt for the LLM to answer the question
3. Invoking the LLM to generate a response
4. Adding the response to the message history in the state
5. Returning the updated state

This simple approach handles basic questions but has limitations - the LLM can only answer based on its training data, without access to current information.

In [12]:
# INSTRUCTOR_COMPLETE_THIS_IN_WORKSHOP
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature = 0.7)

## Initializing the Language Model

We'll use Google's Gemini model as our reasoning engine. LLMs in agent systems serve multiple roles:

1. **Core Reasoning**: Analyzing problems and planning solutions
2. **Tool Use**: Deciding when and how to use available tools
3. **Information Processing**: Extracting and synthesizing knowledge
4. **Response Generation**: Creating human-readable outputs

The choice of LLM impacts the agent's capabilities, with factors like context window size, reasoning abilities, and tool use proficiency all playing important roles.

In [13]:
def is_done(state: AgentState) -> Literal["end"]:
    """Simple function to signal the end of the workflow"""
    return "end"

## Workflow Control Functions

LangGraph uses specialized functions to control the flow of execution through the agent graph.

Here we define `is_done`, a simple function that always returns "end" to signal that processing is complete and the workflow should terminate. In more complex agents, this function could implement conditional logic to determine whether more processing is needed.

In [14]:
# INSTRUCTOR_COMPLETE_THIS_IN_WORKSHOP
# Create the graph with our extended functions
workflow_v1 = StateGraph(AgentState)

# Add our nodes
workflow_v1.add_node("analyze", answer_question)  # New entry point

# Connect process to end
workflow_v1.add_conditional_edges(
    "analyze",
    is_done,
    {
        "end": END,  # Always end after processing
        "continue": "analyze"
    }
)

# Set the entry point to our new analysis node
workflow_v1.set_entry_point("analyze")

# Compile the graph
agent = workflow_v1.compile()

## Creating the Graph Structure

Now we'll build our first graph structure using LangGraph's `StateGraph`. This creates the workflow that our agent will follow.

### Key Components of StateGraph:

1. **Nodes**: Functions that process the state (`analyze_question`)
2. **Edges**: Connections between nodes, defining the flow of execution
3. **Conditional Edges**: Paths that depend on the state or routing functions
4. **Entry Point**: The starting node when the graph is executed

When compiled, the graph becomes a runnable agent that can process user queries through the defined workflow.

In [15]:
query = "What teams were in NBA playoffs in 2025?"
initial_state = AgentState(messages=[HumanMessage(content=query)])

# Run the agent
final_state = agent.invoke(initial_state)

In [16]:
# Display the conversation
for message in final_state['messages']:
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"AI: {message.content}")
    else:
        print(message)

AI: I do not have access to real-time information, including future NBA playoff results. The 2025 NBA playoffs have not yet happened, so the teams that will be participating are not yet known. Once the 2024-2025 regular season concludes, the playoff teams will be determined based on their regular season records.

To give you an idea of how the playoffs work, in the *2024* playoffs, these were the teams that made it:

**Eastern Conference:**

*   (1) Boston Celtics
*   (2) New York Knicks
*   (3) Milwaukee Bucks
*   (4) Cleveland Cavaliers
*   (5) Orlando Magic
*   (6) Indiana Pacers
*   (7) Philadelphia 76ers
*   (8) Miami Heat

**Western Conference:**

*   (1) Oklahoma City Thunder
*   (2) Denver Nuggets
*   (3) Minnesota Timberwolves
*   (4) Los Angeles Clippers
*   (5) Dallas Mavericks
*   (6) Phoenix Suns
*   (7) Los Angeles Lakers
*   (8) New Orleans Pelicans


In [17]:
final_state

{'messages': [AIMessage(content='I do not have access to real-time information, including future NBA playoff results. The 2025 NBA playoffs have not yet happened, so the teams that will be participating are not yet known. Once the 2024-2025 regular season concludes, the playoff teams will be determined based on their regular season records.\n\nTo give you an idea of how the playoffs work, in the *2024* playoffs, these were the teams that made it:\n\n**Eastern Conference:**\n\n*   (1) Boston Celtics\n*   (2) New York Knicks\n*   (3) Milwaukee Bucks\n*   (4) Cleveland Cavaliers\n*   (5) Orlando Magic\n*   (6) Indiana Pacers\n*   (7) Philadelphia 76ers\n*   (8) Miami Heat\n\n**Western Conference:**\n\n*   (1) Oklahoma City Thunder\n*   (2) Denver Nuggets\n*   (3) Minnesota Timberwolves\n*   (4) Los Angeles Clippers\n*   (5) Dallas Mavericks\n*   (6) Phoenix Suns\n*   (7) Los Angeles Lakers\n*   (8) New Orleans Pelicans', additional_kwargs={}, response_metadata={})],
 'search_results': []}

In [ ]:
search_engine = GoogleSearchAPIWrapper()  # Get top 3 results

# ------------------------------------------------------------
# Node 1 – optimise the user query
# ------------------------------------------------------------
def analyze_query(state: AgentState):
    """Analyse the user’s last message and produce an optimised search query."""
    user_query = next(
        (m.content for m in reversed(state.messages)
         if isinstance(m, HumanMessage)),
        ""
    )

    optimisation_prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are an expert at optimising search queries … "
         "Return ONLY the optimised query."),
        ("human", "Optimise this search query: {query}")
    ])

    optimised = llm.invoke(
        optimisation_prompt.format_messages(query=user_query)
    ).content.strip()

    # ✅ Return *only* the update
    return {"optimized_query": optimised}


# ------------------------------------------------------------
# Node 2 – perform the web search
# ------------------------------------------------------------
def perform_search(state: AgentState):
    """Run a web search based on the (possibly) optimised query."""
    search_query = (
        state.optimized_query
        or next(
            (m.content for m in reversed(state.messages)
             if isinstance(m, HumanMessage)),
            ""
        )
    )

    try:
        raw_results = search_engine.results(search_query, num_results=3)
    except Exception as err:
        print(f"Search API error: {err}")
        raw_results = [{
            "title": "Search temporarily unavailable",
            "snippet": f"Could not retrieve results for: {search_query}.",
            "link": "https://example.com"
        }]

    results = [
        SearchResult(
            id=i + 1,
            title=r.get("title", "No title"),
            snippet=r.get("snippet", r.get("description", "No content")),
            link=r.get("link", r.get("url", "No link")),
        )
        for i, r in enumerate(raw_results)
    ]

    # ✅ Return *only* the update (list reducer will concatenate)
    return {"search_results": results}


# ------------------------------------------------------------
# Node 3 – synthesise an answer from search results
# ------------------------------------------------------------
def process_results(state: AgentState):
    """Generate a final answer that cites the search results."""
    from datetime import datetime
    today = datetime.now().strftime("%B %d, %Y")

    # Format results for the LLM
    if not state.search_results:
        results_block = (
            "No search results available. "
            "The search API may be inaccessible."
        )
    else:
        parts = []
        for r in state.search_results:
            parts.append(
                f"Result {r.id}:\n"
                f"Title: {r.title}\n"
                f"Snippet: {r.snippet}\n"
                f"Source: {r.link}\n"
            )
            if r.full_content:
                parts.append(f"Extracted content:\n{r.full_content}\n")
        results_block = "\n".join(parts)

    original_query = next(
        (m.content for m in state.messages if isinstance(m, HumanMessage)),
        "Unknown query"
    )

    synthesis_prompt = ChatPromptTemplate.from_messages([
        ("system",
         f"You are a basketball analyst. Today is {today}.\n"
         "Use the search results to answer precisely, citing dates, scores, etc."),
        ("human",
         "User query: {original_query}\n\nSearch results:\n{search_results}")
    ])

    answer = llm.invoke(
        synthesis_prompt.format_messages(
            original_query=original_query,
            search_results=results_block
        )
    ).content.strip()

    # ✅ Return only the new message; add_messages reducer appends it
    return {"messages": [AIMessage(content=answer)]}


## Enhanced Agent with Web Search Capabilities

To make our agent more powerful, let's add web search capabilities. This transforms it from a system limited by training data to one that can access current information.

### Multi-Step Reasoning Architecture

We'll implement a more sophisticated workflow with multiple specialized components:

1. **Query Analysis**: Optimize the user's question for search effectiveness
2. **Search Execution**: Retrieve relevant information from the web
3. **Result Processing**: Synthesize information into a coherent response

This demonstrates a key pattern in LangGraph: breaking complex tasks into focused subtasks with dedicated reasoning at each step. By specializing each component, we get better performance than trying to accomplish everything in a single prompt.

In [ ]:
# INSTRUCTOR_COMPLETE_THIS_IN_WORKSHOP
def route_to_search(state: AgentState) -> Literal["search"]:
    """Route to the search node after query analysis"""
    return "search"

## Routing Functions in LangGraph

Routing functions determine the flow of execution through the graph. They analyze the current state and return a string identifier that specifies which node should be executed next.

Here, `route_to_search` always returns "search" - creating a simple linear flow from analysis to search. In more complex agents, these functions could implement sophisticated decision logic based on the state.

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
# Create the graph with our extended functions
workflow_v2 = StateGraph(AgentState)

# Set the entry point to our new analysis node
workflow_v2.set_entry_point("analyze")

workflow_v2.add_edge("search", "process")


# Compile the graph
agent_v2 = workflow_v2.compile()
# STUDENT_IMPLEMENTATION_SECTION_END

## Designing the Enhanced Agent Graph

Let's create a more advanced agent graph with our multi-step workflow.

### LangGraph Workflow Design

When designing LangGraph workflows, consider these principles:

1. **Single Responsibility**: Each node should perform a specific, focused task
2. **State Transformation**: Nodes transform the state in predictable ways
3. **Explicit Decision Points**: Use conditional edges to represent decision logic
4. **Error Handling**: Account for failures and edge cases

Our enhanced agent follows this pattern with explicit nodes for query analysis, search, and result processing.

In [ ]:
query = "Who were in 2025 NBA playoffs?"
initial_state = AgentState(messages=[HumanMessage(content=query)])

# Run the agent
final_state = agent_v2.invoke(initial_state)

## Testing Our Enhanced Agent

Let's test our search-enabled agent with a specific query. The execution will follow these steps:

1. Initial state with user query is created
2. Query analysis optimizes the search terms
3. Search retrieves relevant information
4. Processing synthesizes a comprehensive response

This demonstrates the multi-step reasoning pattern in action.

In [ ]:
# This cell shows the full state object - you can run it to explore the complete state
# final_state    

In [ ]:
# Display the conversation
for message in final_state['messages']:
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"AI: {message.content}")
    else:
        print(message)

In [ ]:

search_engine = GoogleSearchAPIWrapper(k=3)

# ── Node 1 ────────────────────────────────────────────────────────────────────
def analyze_query(state: AgentState):
    """Optimise the user’s last message into a tighter search query."""
    user_query = next(
        (m.content for m in reversed(state.messages)
         if isinstance(m, HumanMessage)), ""
    )

    today = datetime.now().strftime("%B %d, %Y")
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         f"You are an expert at optimising search queries.\nToday is {today}. "
         "Return ONLY the optimised query, no explanation."),
        ("human", "Optimise this search query: {query}")
    ])

    optimised = llm.invoke(
        prompt.format_messages(query=user_query)
    ).content.strip().replace('"', '')

    return {"optimized_query": optimised}


# ── Utility (kept as is) ──────────────────────────────────────────────────────
def fetch_full_content(url: str, max_length: int = 8_000) -> str:
    """Fetch full page text with basic error handling."""
    try:
        loader = WebBaseLoader(
            web_paths=[url],
            header_template={"User-Agent":
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/120.0.0.0 Safari/537.36"}
        )
        docs = loader.load()
        if docs and docs[0].page_content.strip():
            text = docs[0].page_content
            return text[:max_length] + ("…" if len(text) > max_length else "")
        return "[Full content unavailable. Using search-result snippet instead]"
    except Exception as e:
        if "timeout" in str(e).lower():
            return "[Content unavailable: Connection timed out]"
        if "403" in str(e) or "forbidden" in str(e).lower():
            return "[Content unavailable: Access forbidden]"
        if "404" in str(e) or "not found" in str(e).lower():
            return "[Content unavailable: Page not found]"
        return f"[Content unavailable: {e}]"


# ── Node 2 ────────────────────────────────────────────────────────────────────
def perform_search(state: AgentState):
    """Run Google search with the (maybe) optimised query."""
    query = state.optimized_query or next(
        (m.content for m in reversed(state.messages)
         if isinstance(m, HumanMessage)), ""
    )

    try:
        raw = search_engine.results(query, num_results=5)
    except Exception as err:
        print(f"Search API error: {err}")
        raw = [{
            "title": "Search temporarily unavailable",
            "snippet": f"Could not retrieve results for: {query}",
            "link": "https://example.com"
        }]

    results = [
        SearchResult(
            id=i + 1,
            title=r.get("title", "No title"),
            snippet=r.get("snippet", r.get("description", "No content")),
            link=r.get("link", r.get("url", "No link"))
        )
        for i, r in enumerate(raw)
    ]

    # list-reducer (`operator.add`) concatenates with existing results
    return {"search_results": results}


# ── Node 3 ────────────────────────────────────────────────────────────────────
def enrich_content(state: AgentState):
    """Fetch each top result and let the LLM extract a concise summary."""
    if not state.search_results:
        return {}        # nothing to do

    user_query = next(
        (m.content for m in state.messages if isinstance(m, HumanMessage)),
        "Unknown query"
    )

    enriched = []
    for result in state.search_results[:2]:      # enrich top 2
        raw = fetch_full_content(result.link)
        if raw.startswith("[Content unavailable"):
            result.full_content = raw
        else:
            ext_prompt = ChatPromptTemplate.from_messages([
                ("system",
                 "Extract the information most relevant to the user’s query "
                 "and summarise in ≤800 words."),
                ("human",
                 "User query: {query}\n\nTitle: {title}\nURL: {url}\n\nContent:\n{content}")
            ])
            summary = llm.invoke(
                ext_prompt.format_messages(
                    query=user_query,
                    title=result.title,
                    url=result.link,
                    content=raw
                )
            ).content.strip()
            result.full_content = summary
        enriched.append(result)

    # replace the first N results with their enriched versions
    merged = enriched + state.search_results[len(enriched):]
    return {"search_results": merged}


# ── Node 4 ────────────────────────────────────────────────────────────────────
def process_results(state: AgentState):
    """Synthesize a final answer citing the enriched search results."""
    today = datetime.now().strftime("%B %d, %Y")
    user_query = next(
        (m.content for m in state.messages if isinstance(m, HumanMessage)),
        "Unknown query"
    )

    if state.search_results:
        blocks = []
        for r in state.search_results:
            blocks.append(
                f"Result {r.id} | {r.title}\n"
                f"{r.snippet}\nSource: {r.link}\n"
                f"{r.full_content or ''}\n"
            )
        results_block = "\n".join(blocks)
    else:
        results_block = (
            "No search results available – the search API may be down."
        )

    synth_prompt = ChatPromptTemplate.from_messages([
        ("system",
         f"You are a research assistant. Today is {today}. "
         "Write a thorough, well-structured answer that cites dates, facts "
         "and includes source attributions."),
        ("human",
         "User query: {query}\n\nSearch results:\n{results}")
    ])

    answer = llm.invoke(
        synth_prompt.format_messages(
            query=user_query,
            results=results_block
        )
    ).content.strip()

    # add_messages reducer appends safely
    return {"messages": [AIMessage(content=answer)]}
# STUDENT_IMPLEMENTATION_SECTION_END

## Advanced Agent with Content Enrichment

### LLMs as Multi-Purpose Components

In modern agent architecture, LLMs serve multiple roles beyond final output generation. Our agent demonstrates this with multiple LLM-powered processes:

1. **Query Analysis and Optimization**: Before searching, the LLM analyzes the user's question to create more effective search terms.

2. **Content Enrichment**: After retrieving search results, the LLM processes web content to extract the most relevant information.

3. **Response Synthesis**: Finally, the LLM combines all information into a coherent, comprehensive answer.

This multi-stage approach is far more effective than attempting to accomplish everything in a single LLM call. The `fetch_full_content` function enables a key capability: retrieving detailed information from web pages to augment the limited snippets provided by search engines.

In [ ]:
# INSTRUCTOR_COMPLETE_THIS_IN_WORKSHOP
def route_to_search(state: AgentState) -> Literal["search"]:
    """Route to the search node after query analysis"""
    return "search"

def should_enrich_content(state: AgentState) -> Literal["enrich", "process"]:
    """Decide whether to enrich content or proceed to processing"""
    # For simplicity in this workshop, we'll always enrich content
    return "enrich"

def is_done(state: AgentState) -> Literal["end"]:
    """Simple function to signal the end of the workflow"""
    return "end"

## Dynamic Flow Control in LangGraph

These functions control the flow of execution in our agent graph:

1. **route_to_search**: Directs the flow from query analysis to search
2. **should_enrich_content**: Determines whether to enrich content or proceed directly to processing
3. **is_done**: Signals the end of the workflow

Conditional routing enables dynamic decision-making in LangGraph. While our current functions are simple, they could implement complex logic in production systems, such as:

- Skipping search for questions the LLM can answer directly
- Enriching only the most promising search results
- Triggering additional research for insufficient information

These decision points are where the "agent" aspect truly emerges - the system adapts its behavior based on the specific context and needs of each query.

In [ ]:
# STUDENT_IMPLEMENTATION_SECTION_BEGIN
# Create the graph with our extended functions
workflow = StateGraph(AgentState)


# Compile the graph
agent = workflow.compile()
# STUDENT_IMPLEMENTATION_SECTION_END

## Building the Complete Agent Workflow

### LangGraph Architecture Patterns

Our complete agent implementation demonstrates key LangGraph architecture patterns:

1. **Graph-Based Orchestration**: Explicit workflow with clearly defined stages
2. **Typed State Management**: Structured state passed and transformed between nodes
3. **Conditional Branching**: Decision points that determine execution flow
4. **Component Specialization**: Each node focused on a specific task

The workflow follows this pattern:
```
[Analyze] → [Search] → [Enrich] → [Process] → END
```

This approach offers several advantages over monolithic agent designs:

- **Maintainability**: Components can be improved independently
- **Transparency**: Reasoning steps can be traced and debugged
- **Specialization**: Prompts optimized for specific subtasks
- **Robustness**: Failure in one component doesn't break the entire system

These patterns are essential for building production-grade agent systems.

In [ ]:
# Initialize with a query about current NBA playoffs
query = "Who won US presidential election in 2024?"
initial_state = AgentState(messages=[HumanMessage(content=query)])

# Run the agent
final_state = agent.invoke(initial_state)

# Display the conversation
for message in final_state['messages']:
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"AI: {message.content}")
    else:
        print(message)

In [ ]:
# This cell shows the final state object - you can run it to explore the complete state
# final_state

## Putting It All Together: Testing the Complete Agent

### The Complete Agent Experience

Let's test our complete agent with a specific query. The execution will demonstrate the full workflow:

1. **Query Analysis**: Optimizing the search terms
2. **Search Execution**: Retrieving information from the web
3. **Content Enrichment**: Processing detailed content from web pages
4. **Response Synthesis**: Creating a comprehensive answer

### Beyond This Workshop

This workshop demonstrates fundamental patterns in LangGraph agent development, but many advanced capabilities are possible:

1. **Memory and Long-term Context**: Storing and retrieving information across sessions
2. **Multi-agent Systems**: Coordinating multiple specialized agents
3. **Planning and Reflection**: Adding meta-cognitive abilities to agents
4. **Feedback and Learning**: Improving agent performance over time
5. **User Adaptation**: Customizing behavior based on user preferences

By understanding the core concepts presented here, you're well-prepared to explore these more advanced topics.

In [ ]:
# Display the final state
final_state

## Workshop Summary and Key Takeaways

### Core Concepts Covered

1. **AI Agent Architecture**
   - Components, state management, and decision making
   - Structuring complex reasoning workflows

2. **LangGraph Framework**
   - StateGraph, nodes, edges, and conditional routing
   - State transformation pattern

3. **Multi-stage LLM Reasoning**
   - Specialized prompting for subtasks
   - Processing and synthesis patterns

4. **Enhanced Agent Capabilities**
   - Web search integration
   - Content enrichment
   - Response synthesis

### Next Steps for Learning

1. **Experiment with different LLMs** to compare their reasoning capabilities
2. **Add more tools** to your agent (e.g., calculators, weather APIs)
3. **Implement persistent memory** to maintain context across sessions
4. **Create testing scenarios** to evaluate your agent's performance
5. **Build domain-specific agents** for particular use cases

By mastering these concepts, you're well on your way to developing sophisticated AI agents that can assist with complex, multi-step tasks while maintaining context and making intelligent decisions.